## Avalanche energy

In [5]:
from dataclasses import dataclass
from typing import Optional
import numpy as np

MU0 = 4e-7 * np.pi
RHO_CU = 1.68e-8  # Ohm m

@dataclass
class AvalancheResult:
    inductive_energy_J: float
    avalanche_time_s: Optional[float]
    avalanche_duty: Optional[float]
    avg_power_W: float
    peak_power_W: Optional[float]

def coil_R_L(side_m: float, area_mm2: float):
    """
    Single-turn square loop, approximate R and L.

    Parameters
    ----------
    side_m : float
        Side length [m] (square loop perimeter = 4*side_m).
    area_mm2 : float
        Copper cross-sectional area [mm^2].

    Returns
    -------
    R : float
        DC resistance [ohm]
    L : float
        Inductance [henry] (crude approximation)
    """
    length = 4.0 * side_m
    A = area_mm2 * 1e-6
    R = RHO_CU * length / A

    # crude inductance approximation for square loop
    r_wire = np.sqrt(A / np.pi)
    a = side_m
    L = 2 * MU0 * a * (np.log(1.08 * a / r_wire) + 0.25)

    return R, L

def avalanche_from_coil(
    *,
    inductance_H: float,
    current_A: float,
    period_s: float,
    clamp_voltage_V: Optional[float] = None,
) -> AvalancheResult:
    """
    Compute avalanche-related quantities for an inductive turn-off.

    Parameters
    ----------
    inductance_H : float
        Coil inductance [H]
    current_A : float
        Current at turn-off [A]
    period_s : float
        System period [s] (e.g. 0.02 for 50 Hz)
    clamp_voltage_V : float, optional
        Avalanche or clamp voltage [V].
        If omitted, avalanche duration and peak power are not computed.

    Returns
    -------
    AvalancheResult
        Dataclass containing energy, timing, duty cycle, and power values.
    """

    # --- Energy per event ---
    E_event = 0.5 * inductance_H * current_A**2

    # --- Average power (repetitive) ---
    P_avg = E_event / period_s

    t_av = None
    duty = None
    P_peak = None

    if clamp_voltage_V is not None:
        # Avalanche duration
        t_av = inductance_H * current_A / clamp_voltage_V

        # Duty cycle
        duty = t_av / period_s

        # Peak instantaneous power during avalanche
        P_peak = clamp_voltage_V * current_A

    return AvalancheResult(
        inductive_energy_J=E_event,
        avalanche_time_s=t_av,
        avalanche_duty=duty,
        avg_power_W=P_avg,
        peak_power_W=P_peak,
    )


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    side_m = 40.0       # side length [m]
    area_mm2 = 10.0      # copper cross-section [mm^2]
    R_coil, coil_L = coil_R_L(side_m, area_mm2)

    result = avalanche_from_coil(
        inductance_H=coil_L,     # 200 µH
        current_A=8.0,         # 100 A
        period_s=0.02,           # 20 ms (50 Hz)
        clamp_voltage_V=800.0,   # MOSFET avalanche / clamp voltage
    )

    print("Avalanche / Inductive Turn-off Results")
    print("-------------------------------------")
    print(f"Induction                  : {coil_L*1e3:.2f}mH")
    print(f"Inductive energy per event : {result.inductive_energy_J:.3f} J")
    print(f"Average power (repetitive) : {result.avg_power_W:.1f} W")

    if result.avalanche_time_s is not None:
        print(f"Avalanche duration         : {result.avalanche_time_s*1e6:.2f} µs")
        print(f"Avalanche duty cycle       : {result.avalanche_duty*100:.4f} %")
        # print(f"Peak avalanche power      : {result.peak_power_W:.0f} W")



Avalanche / Inductive Turn-off Results
-------------------------------------
Induction                  : 1.04mH
Inductive energy per event : 0.033 J
Average power (repetitive) : 1.7 W
Avalanche duration         : 10.40 µs
Avalanche duty cycle       : 0.0520 %
